In [65]:
import praw
import pandas as pd
import datetime as dt
from psaw import PushshiftAPI
import re

In [66]:
reddit = praw.Reddit(client_id='73IApaJarJRRVA',client_secret='QIET4TRTvxlLWDNFfobmDD7oAfM',user_agent='PD sentiment',username='hll4ce',password='Snsh11nE')
api = PushshiftAPI(reddit)

In [71]:
start_epoch = int(dt.datetime(2012, 1, 1).timestamp()) 

submissions_all_time = list(api.search_submissions(after = start_epoch,
                                                   subreddit = 'Parkinsons',
                                                   filter=['url','author', 'title', 'subreddit'],
                                                   limit = None))
len(submissions_all_time)

submissions_dict = { "title":[], "author":[], "score":[], "id":[], "url":[],"comms_num": [], "created": [], "body":[]}
    
for submission in submissions_all_time:
    
    submissions_dict["title"].append(submission.title)
    submissions_dict["author"].append(str(submission.author))
    submissions_dict["score"].append(submission.score)
    submissions_dict["id"].append(submission.id)
    submissions_dict["url"].append(submission.url)
    submissions_dict["comms_num"].append(submission.num_comments)
    submissions_dict["created"].append(submission.created)
    submissions_dict["body"].append(submission.selftext)
    

submissions_data = pd.DataFrame(submissions_dict)

In [73]:
comments_all_time = list(api.search_comments(after = start_epoch,
                                             subreddit = 'Parkinsons',
                                             limit = None))

comments_dict = {"parent_id":[], 
                 "subreddit_id":[],
                 "author":[],
                 "id":[], 
                 "score":[], 
                 "created": [], 
                 "body":[], 
                 "controversiality":[]}
    
for comment in comments_all_time:
    comments_dict["parent_id"].append(comment.parent_id)
    comments_dict["subreddit_id"].append(comment.subreddit_id)
    comments_dict["author"].append(str(submission.author))
    comments_dict["id"].append(comment.id)
    comments_dict["score"].append(comment.score)
    comments_dict["controversiality"].append(comment.controversiality)
    comments_dict["created"].append(comment.created)
    comments_dict["body"].append(comment.body)

comments_data = pd.DataFrame(comments_dict)

In [74]:
submissions_data.head()

,title,author,score,id,url,comms_num,created,body
0,Erectile dysfunction early sign,Russpow,1,ecsetq,https://www.reddit.com/r/Parkinsons/comments/e...,1,1.576787e+09,"Hello, my father currently has Parkinson’s dis..."
1,Loss of concentration,BookPherq,5,ecnwhm,https://www.reddit.com/r/Parkinsons/comments/e...,1,1.576757e+09,I have been living with YOPD for a few years. ...
2,How to detect Parkinson's with certainty?,Dark_Knight003,4,ecif6j,https://www.reddit.com/r/Parkinsons/comments/e...,8,1.576731e+09,"I am 33, Male and have been experiencing rando..."
3,I have a few questions for people with Parkins...,AgentFN2187,7,ec9h83,https://www.reddit.com/r/Parkinsons/comments/e...,6,1.576683e+09,"1#: Do you ever get ""used"" to the shaking? Th..."
4,Looking for advice about levodopa.,None,2,ec5tmc,https://www.reddit.com/r/Parkinsons/comments/e...,2,1.576663e+09,[deleted]


In [75]:
comments_data.head()

,parent_id,subreddit_id,author,id,score,created,body,controversiality
0,t3_ecsetq,t5_2u0be,jamiecrane,fbdfh2h,1,1.576789e+09,"If you are taking Cialis or Viagra, I suspect ...",0
1,t3_ecnwhm,t5_2u0be,jamiecrane,fbde6l5,2,1.576788e+09,You're not alone.\n\nI need a good thriller or...,0
2,t3_ecif6j,t5_2u0be,jamiecrane,fbd7non,1,1.576780e+09,This doesn't sound like Parkinson's but in any...,0
3,t3_ecif6j,t5_2u0be,jamiecrane,fbcmv0p,1,1.576757e+09,Not much in medicine approaches 100% certainty...,0
4,t3_ecif6j,t5_2u0be,jamiecrane,fbci7y2,1,1.576753e+09,"Get thee to a neurologist.\n\nAs others say, t...",0


In [76]:
comments_data["parent_id_tidy"] = [re.sub(".*_", "", x) for x in comments_data["parent_id"]]

In [77]:
# there are some Falses so some ids don't match up from comments to submissions
[x in list(submissions_data["id"]) for x in comments_data["parent_id_tidy"]][1:10]

[True, True, True, True, False, True, True, True, True]

In [78]:
print(len(submissions_data))
print(len(comments_data))

2587
11933


In [79]:
#fix date column
def get_date(created):
    return dt.datetime.fromtimestamp(created)


In [83]:
_timestamp = submissions_data["created"].apply(get_date)

submissions_data = submissions_data.assign(timestamp = _timestamp)

submissions_data.head()

,title,author,score,id,url,comms_num,created,body,timestamp
0,Erectile dysfunction early sign,Russpow,1,ecsetq,https://www.reddit.com/r/Parkinsons/comments/e...,1,1.576787e+09,"Hello, my father currently has Parkinson’s dis...",2019-12-19 20:16:10
1,Loss of concentration,BookPherq,5,ecnwhm,https://www.reddit.com/r/Parkinsons/comments/e...,1,1.576757e+09,I have been living with YOPD for a few years. ...,2019-12-19 12:09:40
2,How to detect Parkinson's with certainty?,Dark_Knight003,4,ecif6j,https://www.reddit.com/r/Parkinsons/comments/e...,8,1.576731e+09,"I am 33, Male and have been experiencing rando...",2019-12-19 04:50:30
3,I have a few questions for people with Parkins...,AgentFN2187,7,ec9h83,https://www.reddit.com/r/Parkinsons/comments/e...,6,1.576683e+09,"1#: Do you ever get ""used"" to the shaking? Th...",2019-12-18 15:35:41
4,Looking for advice about levodopa.,None,2,ec5tmc,https://www.reddit.com/r/Parkinsons/comments/e...,2,1.576663e+09,[deleted],2019-12-18 09:51:35


In [84]:
_timestamp = comments_data["created"].apply(get_date)

comments_data = comments_data.assign(timestamp = _timestamp)

comments_data.head()

,parent_id,subreddit_id,author,id,score,created,body,controversiality,parent_id_tidy,timestamp
0,t3_ecsetq,t5_2u0be,jamiecrane,fbdfh2h,1,1.576789e+09,"If you are taking Cialis or Viagra, I suspect ...",0,ecsetq,2019-12-19 21:01:28
1,t3_ecnwhm,t5_2u0be,jamiecrane,fbde6l5,2,1.576788e+09,You're not alone.\n\nI need a good thriller or...,0,ecnwhm,2019-12-19 20:40:32
2,t3_ecif6j,t5_2u0be,jamiecrane,fbd7non,1,1.576780e+09,This doesn't sound like Parkinson's but in any...,0,ecif6j,2019-12-19 18:23:42
3,t3_ecif6j,t5_2u0be,jamiecrane,fbcmv0p,1,1.576757e+09,Not much in medicine approaches 100% certainty...,0,ecif6j,2019-12-19 11:56:43
4,t3_ecif6j,t5_2u0be,jamiecrane,fbci7y2,1,1.576753e+09,"Get thee to a neurologist.\n\nAs others say, t...",0,ecif6j,2019-12-19 11:01:53


In [86]:
submissions_data.to_csv("/Users/leonardhl/Documents/Sentiment-in-PD-Progression/parkinsons_submissions.csv")
comments_data.to_csv("/Users/leonardhl/Documents/Sentiment-in-PD-Progression/parkinsons_comments.csv")